In [2]:
import boto3
s3=boto3.resource('s3')

In [5]:
s3.Bucket('datasetsgtavautomated').download_file('training_data-1-balanced.npy','training_data-1-balanced.npy')

In [6]:
s3.Bucket('datasetsgtavautomated').download_file('training_data-2-balanced.npy','training_data-2-balanced.npy')

In [7]:
s3.Bucket('datasetsgtavautomated').download_file('training_data-3-balanced.npy','training_data-3-balanced.npy')

In [8]:
s3.Bucket('datasetsgtavautomated').download_file('training_data-4-balanced.npy','training_data-4-balanced.npy')

In [2]:
""" AlexNet.
References:
    - Alex Krizhevsky, Ilya Sutskever & Geoffrey E. Hinton. ImageNet
    Classification with Deep Convolutional Neural Networks. NIPS, 2012.
Links:
    - [AlexNet Paper](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)
"""

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')

    return model

In [3]:
import numpy as np

WIDTH = 160
HEIGHT = 120
LR = 1e-3
EPOCHS = 3
MODEL_NAME = 'pygta5-car-fast-{}-{}-{}-epochs-300K-data.model'.format(LR, 'alexnetv2',EPOCHS)

model = alexnet(WIDTH, HEIGHT, LR)

hm_data = 4
for i in range(EPOCHS):
    for i in range(1,hm_data+1):
        train_data = np.load('training_data-{}-balanced.npy'.format(i), allow_pickle=True)

        train = train_data[:-100]
        test = train_data[-100:]

        X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
        Y = [i[1] for i in train]

        test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
        test_y = [i[1] for i in test]

        model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
            snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

        model.save(MODEL_NAME)


Training Step: 471  | total loss: 1.22567 | time: 2414.194s
| Momentum | epoch: 003 | loss: 1.22567 - acc: 0.3966 -- iter: 09728/12044


KeyboardInterrupt: 

In [ ]:
s3.Bucket('datasetsgtavautomated').upload_file('pygta5-car-fast-0.001-alexnetv2-10-epochs-300K-data.model.data-00000-of-00001','pygta5-car-fast-0.001-alexnetv2-10-epochs-300K-data.model.data-00000-of-00001')

In [ ]:
s3.Bucket('datasetsgtavautomated').upload_file('pygta5-car-fast-0.001-alexnetv2-10-epochs-300K-data.model.index','pygta5-car-fast-0.001-alexnetv2-10-epochs-300K-data.model.index')

In [ ]:
s3.Bucket('datasetsgtavautomated').upload_file('pygta5-car-fast-0.001-alexnetv2-10-epochs-300K-data.model.meta','pygta5-car-fast-0.001-alexnetv2-10-epochs-300K-data.model.meta')